# Jupyter

To get these examples to work in `Jupyter`, you will need to install the following.

- [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/user_install.html)
- [ffmpeg](https://ffmpeg.org/)

## Widgets

Let's see how we can use `zava` to work with `ipywidgets`. First, we got to get some data.

In [1]:
import numpy as np
import pandas as pd

M = np.array([
    [1, 1, 1, 1],
    [2, 2, 2, 1],
    [3, 3, 3, 3],
    [1, 2, 3, 4],
    [2, 2, 1, 1],
    [1, 1, 3, 3]
])

Now we create an instance of `GrandTour` with the data and also specifying the minimum `c` and maximum `d` values for scaling.

In [2]:
from zava.core import GrandTour

c = 0
d = 1
grand_tour = GrandTour(M, c, d)

Finally, we use a function `f` annotated with `@interact` to create an interactive visualization with parallel coordinates and Grand Tour.

In [3]:
import matplotlib.pyplot as plt
from ipywidgets import interact

@interact(degree=(0, 360 * 4, 0.5))
def f(degree=0):
    S = grand_tour.rotate(degree)
    
    fig, ax = plt.subplots(figsize=(15, 3))
    
    params = {
        'kind': 'line',
        'ax': ax,
        'color': 'r',
        'marker': 'h',
        'markeredgewidth': 1,
        'markersize': 5,
        'linewidth': 0.8
    }
    
    _ = S.plot(**params)
    _ = ax.get_legend().remove()
    _ = ax.set_xticks(np.arange(len(S.index)))
    _ = ax.set_xticklabels(S.index)
    _ = ax.get_yaxis().set_ticks([])
    _ = ax.set_title('Grand Tour')

interactive(children=(FloatSlider(value=0.0, description='degree', max=1440.0, step=0.5), Output()), _dom_clas…

## Animations

Now let's see how we can create HTML5 animations in a notebook using `matplotlib.animation`. Again, start with some data.

In [4]:
import numpy as np
import pandas as pd

M = np.array([
    [1, 1, 1, 1],
    [2, 2, 2, 1],
    [3, 3, 3, 3],
    [1, 2, 3, 4],
    [2, 2, 1, 1],
    [1, 1, 3, 3]
])

Create a `GrandTour` instance with the data.

In [5]:
from zava.core import GrandTour

c = 0
d = 1
grand_tour = GrandTour(M, c, d)

We have to wrap the `GrandTour` instance with a `SinglePlotter`. The `SinglePlotter` plots only a single set of data with an `axis` and does not concern itself with the greater plot (e.g. the title). The `params` argument is a dictionary that you can override to change the line drawings.

In [6]:
from zava.plot import SinglePlotter

single_plotter = SinglePlotter(grand_tour, params={'color': 'r'})

The `MultiPlotter` controls all the plots and takes in a list of `SinglePlotters` as well as an `axis`. You can then use an instance of this object with `animation.FuncAnimation()` as usual to produce an animation.

In [7]:
from zava.plot import MultiPlotter
from matplotlib import animation

fig, ax = plt.subplots(figsize=(5, 3))

multi_plotter = MultiPlotter([single_plotter], ax=ax)

params = {
    'fig': fig,
    'func': multi_plotter,
    'frames': np.linspace(0, 360, 360),
    'interval': 20,
    'init_func': multi_plotter.init
}
anim = animation.FuncAnimation(**params)

plt.close(fig)

Finally, render the video.

In [8]:
%%time

from IPython.display import HTML

HTML(anim.to_html5_video())

CPU times: user 21.4 s, sys: 574 ms, total: 22 s
Wall time: 22.1 s


## Animation, colors

You might find yourself doing cluster analysis of high-dimensional data. If you recover some clusters, you can break the data apart according to the clusters and visualize them with different colors. Here's a full working example (without the clustering) of how to visualize two datasets.

In [9]:
%%time

# 1. here are your two datasets, M1 and M2

columns = ['v0', 'v1', 'v2', 'v3']

M1 = np.array([
    [1, 1, 1, 1],
    [2, 2, 2, 1],
    [3, 3, 3, 3]
])
M2 = np.array([
    [1, 2, 3, 4],
    [2, 2, 1, 1],
    [1, 1, 3, 3]
])

M1 = pd.DataFrame(M1, columns=columns)
M2 = pd.DataFrame(M2, columns=columns)

# 2. create your GrandTour instances

c = 0.0
d = 100.0

gt1 = GrandTour(M1, c, d)
gt2 = GrandTour(M2, c, d)

# 3. create corresponding SinglePlotters

sp1 = SinglePlotter(gt1, params={'color': 'r'})
sp2 = SinglePlotter(gt2, params={'color': 'g'})

# 4. create a MultiPlotter from the SinglePlotters
fig, ax = plt.subplots(figsize=(5, 3))
mp = MultiPlotter([sp1, sp2], ax=ax)

params = {
    'fig': fig,
    'func': mp,
    'frames': np.linspace(0, 360, 360),
    'interval': 20,
    'init_func': mp.init
}
anim = animation.FuncAnimation(**params)

plt.close(fig)

# 5. display the animation
HTML(anim.to_html5_video())

CPU times: user 25 s, sys: 608 ms, total: 25.6 s
Wall time: 25.7 s
